# Preliminaries: building a *k*-nearest neighbours classifier

In [the previous Notebook](22.1 Case study preliminaries - the vector space model.ipynb) we saw how to represent a collection of documents as term frequency vectors using Python, and how cosine distance can be used to estimate the similarity between pairs of documents.

In this Notebook we will build a simple *k*-nearest neighbours classifier which will form the basis of our spam filter. Be aware that in these Notebooks we are *not* attempting to make the code as efficient as possible. Rather, we are aiming to show how an intuitively simple algorithm, such as *k*-NN, can be used to deliver good results on a practical, real-world application. If you were to try to use these techniques on a genuinely large dataset (such as on all the email sent and received by a large corporation in a typical day), you would expect to need a significant amount of time exploring techniques to optimise your implementations of the algorithms (or, of course, to use an off-the-shelf implementation). 

You should spend around an hour on this Notebook, and around a further 40 minutes on the iCMA questions referenced at the end.

## Initial imports and function definitions

In this Notebook the functions we use are just the same as those defined in [Notebook 22.1](22.1 Case study preliminaries - the vector space model.ipynb). We will also be using the `KNeighborsClassifier` library which we explored in Part 20.

In [ ]:
# Standard imports
import pandas as pd

import math

from scipy.spatial.distance import cosine
from sklearn.neighbors import KNeighborsClassifier

from collections import Counter

The following function definitions are exactly those which we developed in [Notebook 22.1](22.1 Case study preliminaries - the vector space model.ipynb). If you need a reminder of how they are used, you should reread that Notebook, in particular the section entitled 'Bringing it all together: estimating document similarity'.

In [ ]:
def tokenise_document(docIn_str):
    '''Return a list of the tokens in the input string docIn_str'''
    return docIn_str.split()

In [ ]:
def build_term_index(tokenisedDocuments_coll):
    '''Return a set of all the terms appearing in the 
       documents in tokenisedDocuments_coll
    '''
    allTerms_set = set()  # Store the tokens as a set to remove repetitions
    
    for tokens_coll in tokenisedDocuments_coll:
        allTerms_set = allTerms_set.union(set(tokens_coll))
        
    return list(allTerms_set)     # Return the members as a list

In [ ]:
def build_tf_vector(tokenisedDocument_ls, termIndex_ls):
    '''Return a pandas Series representing the term 
       frequency vector of the tokenised document 
       tokenisedDocument_ls, and indexed with termIndex_ls
    '''
    
    return pd.Series(Counter(tokenisedDocument_ls),
                     index=termIndex_ls).fillna(0)
    

## Defining some toy data

To provide some initial data to help us build the classifier, we will use some invented sentences, which can be divided into two categories: *ham* and *spam*. The aim of the classifier will be to estimate whether a new document is more similar to the *ham* documents, or to the *spam* documents.

Ham documents:
    
1. *details of the meeting will be sent by email*

2. *we anticipate a big financial loss*

3. *we need to reduce the size of the workforce*

4. *the financial meeting aims to win over investors*

Spam documents:
    
5. *reduce your hair loss now*

6. *enter the prize draw and win big*

7. *email us for details of this unique opportunity*

8. *your bank account will be suspended*

As in the previous Notebook, we will represent each of these documents using a Python dictionary, where the keys in the dictionaries are terms, and the values are counts of the term in the document.

We will also set up two lists of values: the first will contain the documents themselves, and the second the classes of those documents. Then the *n*th member of the list of documents will have the class of the *n*th member of the list of classes.

In [ ]:
trainingDocuments_ls = ['details of the meeting will be sent by email',
                        'we anticipate a big financial loss',                 
                        'we need to reduce the size of the workforce',                
                        'the financial meeting aims to win over investors',
                        'reduce your hair loss now', 
                        'enter the prize draw and win big',
                        'email us for details of this unique opportunity',                 
                        'your bank account will be suspended']

And set up another list to represent the document classes:

In [ ]:
trainingClasses_ls = ['ham', 'ham', 'ham', 'ham', 'spam', 'spam', 'spam', 'spam']

## Building a DataFrame of training data

When you worked through Notebook `20.1 The k-nearest neighbours classifier` you saw that we can use a *pandas* DataFrame as the input to the classifier. In this section of the Notebook we will build a DataFrame of the training documents in the same way.

What we are aiming for is a DataFrame whose rows represent the training documents and whose columns represent the documents' features: in this case, the terms in each document. So we are aiming for a DataFrame which looks like:

||details|of|we|anticipate| ... |
|:---:|:---:|:---:|:---:|:---:|:---:|
|**doc 1**|1|1|0|0| ... |
|**doc 2**|0|0|1|1| ... |
| $\vdots$ | $\vdots$ | $\vdots$ | $\vdots$ | $\vdots$ ||

Because each column of a DataFrame object is a Series object, it is straightforward to build the training data DataFrame from the Series objects returned by the `build_tf_vector` function. 

Following the same procedure as in [Notebook 22.1](22.1 Case study preliminaries - the vector space model.ipynb), our first task is to convert the list of training documents into a list of tokenised documents:

In [ ]:
tokenisedTrainingDocuments_ls = [tokenise_document(doc_str) for doc_str in trainingDocuments_ls]

The *n*th member of `tokenisedTrainingDocuments_ls` is a tokenised form of the *n*th member of `trainingDocuments_ls`. For example, choosing an arbitrary member of the lists:

In [ ]:
n = 3   # arbitrary choice, can have 0 <= n <= 7
print(trainingDocuments_ls[n])
print()
print(tokenisedTrainingDocuments_ls[n])

As before, we use the `build_term_index` function and the collection of tokenised documents to build a term index:

In [ ]:
termIndex_ls = build_term_index(tokenisedTrainingDocuments_ls)

and use this to convert the tokenised documents to term vectors:

In [ ]:
trainingTfVectors_ls = [build_tf_vector(tokenisedDoc_ls, termIndex_ls)
                        for tokenisedDoc_ls in tokenisedTrainingDocuments_ls]

Now that the documents are represented as a list of term vectors, we can convert these to a DataFrame by simply calling the `DataFrame` constructor on the list of term vectors:

In [ ]:
trainingData_df = pd.DataFrame(trainingTfVectors_ls)

trainingData_df

in which the *n*th row encodes the term vector representing the *n*th document.

## Training the classifier

We can now train the classifier in the same was as in Notebook 20.1. As before, we will use a *k*-NN classifier, with, in this case, *k*=3:

In [ ]:
spamFilter3_knn = KNeighborsClassifier(n_neighbors=3, metric='cosine', algorithm='brute')

In this case, we have set the metric to `cosine` so that the algorithm will use cosine distance rather than Euclidean distance. The option `algorithm='brute'` is a quirk which tells the learner to use the (least efficient) brute-force algorithm in training the classifier: there are various algorithms which improve the training efficiency for Euclidean distance which cannot be used for cosine distance, and so this option is simply telling the learner not to use them.

In [ ]:
spamFilter3_knn.fit(trainingData_df,
                    trainingClasses_ls)

## Using the classifier to classify test data

Now that we have a trained classifier, we can use it to attempt to classify some new documents. Suppose we have the following documents:

1. *the investors will email this opportunity to the workforce*
2. *please email us your bank details to win*

We would hope that the spam filter might classify the first example as ham, and the second as spam.

To classify these cases, we need to construct a further DataFrame containing the test data. We follow the same sequence as before (although we use the same term index for the test data as the training data, to ensure that the term vectors align).

First, create a collection of test documents:

In [ ]:
testDocuments_ls = ['the investors will email this opportunity to the workforce',
                    'please email us your bank details to win']

Next, tokenise the test documents:

In [ ]:
testTokenisedDocuments_ls = [tokenise_document(testDoc_str)
                             for testDoc_str in testDocuments_ls]

Next, convert the tokenised test documents to term frequency vectors:

In [ ]:
testTfVectors_ls = [build_tf_vector(testDoc_str, termIndex_ls)
                    for testDoc_str in testTokenisedDocuments_ls]

and finally, convert the term vectors into a DataFrame:

In [ ]:
testData_df = pd.DataFrame(testTfVectors_ls)

testData_df

We can now apply the trained classifier to this data to see how the test data is classified.

In [ ]:
spamFilter3_knn.predict(testData_df)

So the first document has been classified as ham, and the second as spam, as we would have hoped.

We have now built a simple, but working, spam filter which uses a collection of classified ham and spam documents to attempt to determine whether a new document is spam or not.

## What next?

You are now ready to attempt Questions 5 and 6 of iCMA 46, which cover the material you have seen in this Notebook and the previous one. You should expect to spend around 10 minutes on Question 5, and around half an hour on Question 6.

If you are working through this Notebook as part of an inline exercise, return to the module materials now.

If you are working through this set of Notebooks as a whole, move on to [`22.3 Applying the classifier to a real dataset`](22.3 Applying the classifier to a real dataset.ipynb).